# Comparison of London and Madrid Venues for Tourists and Business Startups

Author - M / 01.02.21

# 1. Introduction

The aim of this review is to compare London and Madrid, specifically the most frequent venues and the differences between the two Cities. This study will aim to identify data clusters to enable a reasonable statistical comparison. 

This information could be interesting to people trying to choose whether to travel to Madrid of London and to provide some information on the possible niches for venues/businesses that could be established.  

# 2. Data

The data for this study will be acquired from the public domain. Specifically city regional data extracted from Wikipedia and Venue data extracted from Foursquare. 

1. The data from Wikipedia will provide a framework for Geographical data. 
2. The data from Wikipedia with be georeferenced using ArcGIS Geocding
3. The Geocoded data will be used to extract venue information from FourSquare. 
4. The data from Foursquare will be used to define the venues and used in this analysis. 

All the data is freely available with FourSquare accessed using a Free Developer API. 

Information, code and findings will be presented in a Python, Jupyter Notebook format. 

The pandas, requests, nupy, matplotlib, folium, sklearn, arcgis packages were used in this analysis. 

# 3 Method

The methodology will be outlined below with the following sequence / headers

Jupyter notebook and Python shall be used. Various packages were presented in the following section adn will be presented as code below. 

A methodology summary is as follows: Extract city (London and Madrid) zone data from Wikipedia, process data in python and ensure data is in suitable format for further analysis, extract geolocation, use FourSquare to extract venue data and combine with the zone data from wikipedia, use kmeans statistical analysis to generate clusters and extract data. Compare clustered venue data. 

*Outline
3.1 Importation of Python data packages
3.2 London
3.2.1 Data Scraping
3.2.2 Data - Area List Processing 
3.2.3 Data - Geolocating and Map Visualisation
3.2.4 Data - Venue Extraction and Grouping 
3.2.5 Modelling - K Means 
3.2.6 Data Analyis
3.3 Madrid 
3.3.1 Data Scraping
3.3.2 Data - Area List Processing 
3.3.3 Data - Geolocating and Map Visualisation
3.3.4 Data - Venue Extraction and Grouping 
3.3.5 Modelling - K Means 
3.3.6 Data Analyis

3.1 Import Python Data Packages

In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
from sklearn.cluster import KMeans
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()

# 3.2 London Data Analysis

3.2.1 London Data Scraping

In [2]:
url_ldn = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
wiki_ldn_url = requests.get(url_ldn)
wiki_ldn_url

<Response [200]>

In [3]:
wiki_ldn_data = pd.read_html(wiki_ldn_url.text)
wiki_ldn_data

[                                                   0
 0  Map all coordinates in "Category:Areas of Lond...
 1                 Download coordinates as: KML · GPX,
             Location                     London borough       Post town  \
 0         Abbey Wood              Bexley, Greenwich [7]          LONDON   
 1              Acton  Ealing, Hammersmith and Fulham[8]          LONDON   
 2          Addington                         Croydon[8]         CROYDON   
 3         Addiscombe                         Croydon[8]         CROYDON   
 4        Albany Park                             Bexley  BEXLEY, SIDCUP   
 ..               ...                                ...             ...   
 526         Woolwich                          Greenwich          LONDON   
 527   Worcester Park       Sutton, Kingston upon Thames  WORCESTER PARK   
 528  Wormwood Scrubs             Hammersmith and Fulham          LONDON   
 529          Yeading                         Hillingdon           HAYES   
 

In [4]:
wiki_ldn_data = wiki_ldn_data[1]
wiki_ldn_data

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


3.2.2 Data List Processing

In [7]:
wiki_ldn_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_ldn_data

,Location,London borough,Post_town,Postcode district,Dial code,OS_grid_ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728
...,...,...,...,...,...,...
526,Woolwich,Greenwich,LONDON,SE18,020,TQ435795
527,Worcester Park,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4,020,TQ225655
528,Wormwood Scrubs,Hammersmith and Fulham,LONDON,W12,020,TQ225815
529,Yeading,Hillingdon,HAYES,UB4,020,TQ115825


In [5]:
df1 = wiki_ldn_data.drop( [ wiki_ldn_data.columns[0], wiki_ldn_data.columns[4], wiki_ldn_data.columns[5] ], axis=1)

In [6]:
df1.head()

,London borough,Post town,Postcode district
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [7]:
df1.columns = ['Zone','City','PC']
df1

,Zone,City,PC
0,"Bexley, Greenwich [7]",LONDON,SE2
1,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4"
2,Croydon[8],CROYDON,CR0
3,Croydon[8],CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"
...,...,...,...
526,Greenwich,LONDON,SE18
527,"Sutton, Kingston upon Thames",WORCESTER PARK,KT4
528,Hammersmith and Fulham,LONDON,W12
529,Hillingdon,HAYES,UB4


In [8]:
df1['Zone'] = df1['Zone'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

In [9]:
df1.head()

,Zone,City,PC
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
2,Croydon,CROYDON,CR0
3,Croydon,CROYDON,CR0
4,Bexley,"BEXLEY, SIDCUP","DA5, DA14"


In [13]:
df1.shape

(531, 3)

In [10]:
df1 = df1[df1['City'].str.contains('LONDON')]
df1

,Zone,City,PC
0,"Bexley, Greenwich",LONDON,SE2
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4"
6,City,LONDON,EC3
7,Westminster,LONDON,WC2
9,Bromley,LONDON,SE20
...,...,...,...
521,Redbridge,LONDON,"IG8, E18"
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8
525,Barnet,LONDON,N12
526,Greenwich,LONDON,SE18


In [11]:
df1.shape

(308, 3)

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308 entries, 0 to 528
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Zone    308 non-null    object
 1   City    308 non-null    object
 2   PC      308 non-null    object
dtypes: object(3)
memory usage: 9.6+ KB


3.2.3 Data - Geolocating and Map Visualisation

In [13]:
def get_x_y_uk(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, London, England'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [14]:
geo_xy_uk = df1['PC']    
geo_xy_uk

0           SE2
1        W3, W4
6           EC3
7           WC2
9          SE20
         ...   
521    IG8, E18
522         IG8
525         N12
526        SE18
528         W12
Name: PC, Length: 308, dtype: object

In [15]:
lalnldn = geo_xy_uk.apply(lambda x: get_x_y_uk(x))
lalnldn

0       51.492450000000076,0.12127000000003818
1        51.51324000000005,-0.2674599999999714
6       51.51200000000006,-0.08057999999994081
7       51.51651000000004,-0.11967999999995982
9       51.41009000000008,-0.05682999999993399
                        ...                   
521    51.589770000000044,0.030520000000024083
522      51.50642000000005,-0.1272099999999341
525     51.615920000000074,-0.1767399999999384
526      51.48207000000008,0.07143000000002075
528      51.50645000000003,-0.2369099999999662
Name: PC, Length: 308, dtype: object

In [19]:
laldn = lalnldn.apply(lambda x: x.split(',')[0])
laldn

0      51.492450000000076
1       51.51324000000005
6       51.51200000000006
7       51.51651000000004
9       51.41009000000008
              ...        
521    51.589770000000044
522     51.50642000000005
525    51.615920000000074
526     51.48207000000008
528     51.50645000000003
Name: PC, Length: 308, dtype: object

In [20]:
lnldn = lalnldn.apply(lambda x: x.split(',')[1])
lnldn

0       0.12127000000003818
1       -0.2674599999999714
6      -0.08057999999994081
7      -0.11967999999995982
9      -0.05682999999993399
               ...         
521    0.030520000000024083
522     -0.1272099999999341
525     -0.1767399999999384
526     0.07143000000002075
528     -0.2369099999999662
Name: PC, Length: 308, dtype: object

Merging Lat and Long datasets and merging the coordinates with the category data

In [21]:
ldn_mrg = pd.concat([df1,laldn.astype(float), lnldn.astype(float)], axis=1)
ldn_mrg.columns= ['Zone','City','PC','latitude','longitude']
ldn_mrg

,Zone,City,PC,latitude,longitude
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746
6,City,LONDON,EC3,51.51200,-0.08058
7,Westminster,LONDON,WC2,51.51651,-0.11968
9,Bromley,LONDON,SE20,51.41009,-0.05683
...,...,...,...,...,...
521,Redbridge,LONDON,"IG8, E18",51.58977,0.03052
522,"Redbridge, Waltham Forest","LONDON, WOODFORD GREEN",IG8,51.50642,-0.12721
525,Barnet,LONDON,N12,51.61592,-0.17674
526,Greenwich,LONDON,SE18,51.48207,0.07143


In [22]:
ldn_mrg.dtypes

Zone          object
City          object
PC            object
latitude     float64
longitude    float64
dtype: object

In [24]:
ldn = geocode(address='London, England')[0]
ldn_ln_coords = ldn['location']['x']
ldn_la_coords = ldn['location']['y']

# Map of London with Neighbourhoods 

In [55]:
map_Ldn = folium.Map(location=[london_lat_coords, london_lng_coords], zoom_start=12)
map_Ldn

for latitude, longitude, Zone, City in zip(ldn_mrg['latitude'], ldn_mrg['longitude'], ldn_mrg['Zone'], ldn_mrg['City']):
    label = '{}, {}'.format(City, Zone)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_Ldn)  
    
map_Ldn

3.2.4 Data - Venue Extraction and Grouping 

In [26]:
CLIENT_ID = 'DDJ52YS2BFQJWYWRIMTCIYEERAAHV3KRB5CE3SSHRAW4CDCU' 
CLIENT_SECRET = 'MKF5M1HHI3XHYMYFJ0STACUUAIJ31HTTS1Y3P5K4BMJEBCDO'
VERSION = '20180605'

In [144]:
LIMIT=50

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
ldnvenue = getNearbyVenues(ldn_mrg['Zone'], ldn_mrg['latitude'], ldn_mrg['longitude'])

Bexley, Greenwich 
Ealing, Hammersmith and Fulham
City
Westminster
Bromley
Islington
Islington
Barnet
Enfield
Wandsworth
Southwark
City
Richmond upon Thames
Barnet
Islington
Wandsworth
Westminster
Bromley
Newham
Ealing
Westminster
Lewisham
Camden
Southwark
Tower Hamlets
Bexley
City
Lewisham
Greenwich
Tower Hamlets
Camden
Haringey
Tower Hamlets
Haringey
Barnet
Brent
Lambeth
Lewisham
Tower Hamlets
Kensington and Chelsea, Hammersmith and Fulham
Brent
Barnet
Barnet
Southwark
Tower Hamlets
Camden
Tower Hamlets
Waltham Forest
Newham
Islington
Richmond upon Thames
Lewisham
Camden
Westminster
Greenwich
Kensington and Chelsea
Barnet
Westminster
Lewisham
Waltham Forest
Hounslow, Ealing, Hammersmith and Fulham
Brent
Barnet
Lambeth, Wandsworth
Islington
Barnet
Merton
Barnet
Westminster
Barnet, Brent, Camden
Lewisham
Bexley
Haringey
Bromley
Tower Hamlets
Newham
Hackney
Islington
Southwark
Lewisham
Brent
Southwark
Ealing
Kensington and Chelsea
Wandsworth
Southwark
Barnet
Newham
Richmond upon Thames


In [29]:
ldnvenue.head()
#Data QC

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,"Bexley, Greenwich",51.49245,0.12127,Lesnes Abbey,Historic Site
1,"Bexley, Greenwich",51.49245,0.12127,Sainsbury's,Supermarket
2,"Bexley, Greenwich",51.49245,0.12127,Lidl,Supermarket
3,"Bexley, Greenwich",51.49245,0.12127,Abbey Wood Railway Station (ABW),Train Station
4,"Bexley, Greenwich",51.49245,0.12127,Bean @ Work,Coffee Shop


In [30]:
ldnvenue.shape
#Data QC

(8758, 5)

In [31]:
ldnvenue.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Adult Boutique,Islington,51.52969,-0.08697,Sh! Women's Erotic Emporium
African Restaurant,Westminster,51.52587,-0.08808,Red Sea Restaurant
American Restaurant,Waltham Forest,51.61780,0.02795,Spielburger
Antique Shop,Kensington and Chelsea,51.51244,-0.20639,Alice's
Arcade,Westminster,51.51651,-0.11968,Novelty Automation
...,...,...,...,...
Wings Joint,Hammersmith and Fulham,51.54187,-0.19795,Wingmans
Women's Store,"Kensington and Chelsea, Hammersmith and Fulham",51.55457,-0.11478,Vivien of Holloway
Xinjiang Restaurant,Southwark,51.47480,-0.09313,Silk Road


In [34]:
ldnvenuecat = pd.get_dummies(ldnvenue[['Venue Category']], prefix="", prefix_sep="")
ldnvenuecat

,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
ldnvenuecat['Neighbourhood'] = ldnvenue['Neighbourhood'] 

fixed_columns = [ldnvenuecat.columns[-1]] + list(ldnvenuecat.columns[:-1])
ldnvenuecat = ldnvenuecat[fixed_columns]

ldnvenuecat.head()

,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Bexley, Greenwich",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Mean values of Categories -  Mean of venue catagories calculated by Neighbourhood - LDN

In [116]:
ldnall = ldnvenuecat.groupby('Neighbourhood').mean().reset_index()
ldnall.head()

,Neighbourhood,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,...,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo Exhibit
0,Barnet,0.0,0.0,0.001795,0.0,0.0,0.0,0.007181,0.0,0.0,...,0.007181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Barnet, Brent, Camden",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bexley,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bexley, Greenwich",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [118]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create Top Venue Categories - LDN

In [131]:
ldnvensrtnh = pd.DataFrame(columns=columns)
ldnvensrtnh['Neighbourhood'] = ldnall ['Neighbourhood']

for ind in np.arange(ldnall.shape[0]):
    ldnvensrtnh.iloc[ind, 1:] = return_most_common_venues(ldnall.iloc[ind, :], num_top_venues)

ldnvensrtnh.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barnet,Coffee Shop,Café,Grocery Store,Pub,Pharmacy,Supermarket,Italian Restaurant,Bus Stop,Sushi Restaurant,Turkish Restaurant
1,"Barnet, Brent, Camden",Bus Station,Clothing Store,Gym / Fitness Center,Hardware Store,Supermarket,Film Studio,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2,Bexley,Supermarket,Historic Site,Platform,Convenience Store,Coffee Shop,Train Station,Golf Course,Bus Stop,Park,Exhibit
3,"Bexley, Greenwich",Daycare,Home Service,Sports Club,Park,Golf Course,Construction & Landscaping,Historic Site,Bus Stop,Discount Store,Exhibit
4,"Bexley, Greenwich",Supermarket,Platform,Convenience Store,Coffee Shop,Historic Site,Train Station,Zoo Exhibit,Exhibit,Falafel Restaurant,Farmers Market


3.2.5 Modelling - K Means 

In [132]:
k_num_clusters = 6

ldnallclstr = ldnall.drop('Neighbourhood', 1)

ldnkm = KMeans(n_clusters=k_num_clusters, random_state=0).fit(ldnallclstr)
ldnkm

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

Labelling dataset

In [133]:
ldnkm.labels_

array([0, 4, 3, 0, 3, 0, 0, 0, 5, 0, 0, 0, 5, 5, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 5, 2, 0, 0, 5, 0, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5,
       0, 0, 0, 0, 0, 5])

In [134]:
ldnvensrtnh.insert(0, 'Cluster Labels', ldnkm.labels_ +1)

In [135]:
ldndata = ldn_mrg

ldndata = ldndata.join(ldnvensrtnh.set_index('Neighbourhood'), on='Zone')

ldndata.head()

,Zone,City,PC,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bexley, Greenwich",LONDON,SE2,51.49245,0.12127,4,Supermarket,Platform,Convenience Store,Coffee Shop,Historic Site,Train Station,Zoo Exhibit,Exhibit,Falafel Restaurant,Farmers Market
1,"Ealing, Hammersmith and Fulham",LONDON,"W3, W4",51.51324,-0.26746,2,Grocery Store,Indian Restaurant,Train Station,Park,Breakfast Spot,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fishing Store
6,City,LONDON,EC3,51.51200,-0.08058,6,Hotel,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Pub,Garden,Wine Bar,Falafel Restaurant,Restaurant,Scenic Lookout
7,Westminster,LONDON,WC2,51.51651,-0.11968,6,Hotel,Coffee Shop,Theater,Pub,Burger Joint,Café,Juice Bar,Italian Restaurant,French Restaurant,Hotel Bar
9,Bromley,LONDON,SE20,51.41009,-0.05683,1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Café,Park,Bus Stop,Gastropub,Bistro


In [136]:
ldndatanan = ldndata.dropna(subset=['Cluster Labels'])

3.2.6 Data Analyis

# Plotting the data on a map - LDN

In [137]:
map_clusters_london = folium.Map(location=[ldn_ln_coords, ldn_la_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ldndata['latitude'], ldndata['longitude'], ldndata['Zone'], ldndata['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_london)
        
map_clusters_london

Cluster Set 1

In [138]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 1, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,LONDON,1,Supermarket,Convenience Store,Hotel,Fast Food Restaurant,Grocery Store,Café,Park,Bus Stop,Gastropub,Bistro
14,"BARNET, LONDON",1,Coffee Shop,Café,Grocery Store,Pub,Pharmacy,Supermarket,Italian Restaurant,Bus Stop,Sushi Restaurant,Turkish Restaurant
15,LONDON,1,Italian Restaurant,Coffee Shop,Supermarket,Grocery Store,Pizza Place,Pub,Sandwich Place,Fast Food Restaurant,Café,Turkish Restaurant
16,LONDON,1,Coffee Shop,Pub,Indian Restaurant,Supermarket,Café,Portuguese Restaurant,Grocery Store,Bar,Pizza Place,Gym
17,LONDON,1,Pub,Café,Coffee Shop,Park,Grocery Store,Gym / Fitness Center,Bar,Italian Restaurant,Pharmacy,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...,...
520,LONDON,1,Pub,Café,Coffee Shop,Grocery Store,Pizza Place,Park,Food & Drink Shop,Japanese Restaurant,Bakery,Hardware Store
521,LONDON,1,Grocery Store,Café,Coffee Shop,Pub,Chinese Restaurant,Bar,BBQ Joint,Park,Bakery,Metro Station
525,LONDON,1,Coffee Shop,Café,Grocery Store,Pub,Pharmacy,Supermarket,Italian Restaurant,Bus Stop,Sushi Restaurant,Turkish Restaurant
526,LONDON,1,Pub,Grocery Store,Bus Stop,Indian Restaurant,Coffee Shop,Historic Site,Park,Pier,Fish & Chips Shop,Golf Course


Cluster Set 2

In [139]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 2, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,LONDON,2,Grocery Store,Indian Restaurant,Train Station,Park,Breakfast Spot,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fishing Store


Cluster 3

In [140]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 3, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
377,"HARROW, STANMOREEDGWARE, LONDON",3,Bakery,Chinese Restaurant,Gym,Fish & Chips Shop,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio


Cluster 4

In [141]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 4, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,LONDON,4,Supermarket,Platform,Convenience Store,Coffee Shop,Historic Site,Train Station,Zoo Exhibit,Exhibit,Falafel Restaurant,Farmers Market
45,"BEXLEYHEATH, LONDON",4,Supermarket,Historic Site,Platform,Convenience Store,Coffee Shop,Train Station,Golf Course,Bus Stop,Park,Exhibit
124,LONDON,4,Supermarket,Historic Site,Platform,Convenience Store,Coffee Shop,Train Station,Golf Course,Bus Stop,Park,Exhibit
291,"LONDON, SIDCUP",4,Supermarket,Historic Site,Platform,Convenience Store,Coffee Shop,Train Station,Golf Course,Bus Stop,Park,Exhibit
505,LONDON,4,Supermarket,Historic Site,Platform,Convenience Store,Coffee Shop,Train Station,Golf Course,Bus Stop,Park,Exhibit


Cluster 5

In [142]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 5, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
121,LONDON,5,Bus Station,Clothing Store,Gym / Fitness Center,Hardware Store,Supermarket,Film Studio,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant


Cluster 6

In [143]:
ldndatanan.loc[ldndatanan['Cluster Labels'] == 6, ldndatanan.columns[[1] + list(range(5, ldndatanan.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,LONDON,6,Hotel,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Pub,Garden,Wine Bar,Falafel Restaurant,Restaurant,Scenic Lookout
7,LONDON,6,Hotel,Coffee Shop,Theater,Pub,Burger Joint,Café,Juice Bar,Italian Restaurant,French Restaurant,Hotel Bar
10,LONDON,6,Coffee Shop,Pub,Café,Vietnamese Restaurant,Hotel,Cocktail Bar,Italian Restaurant,Food Truck,Gym / Fitness Center,Breakfast Spot
12,LONDON,6,Coffee Shop,Pub,Café,Vietnamese Restaurant,Hotel,Cocktail Bar,Italian Restaurant,Food Truck,Gym / Fitness Center,Breakfast Spot
18,LONDON,6,Hotel,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Pub,Garden,Wine Bar,Falafel Restaurant,Restaurant,Scenic Lookout
26,LONDON,6,Coffee Shop,Pub,Café,Vietnamese Restaurant,Hotel,Cocktail Bar,Italian Restaurant,Food Truck,Gym / Fitness Center,Breakfast Spot
28,LONDON,6,Hotel,Coffee Shop,Theater,Pub,Burger Joint,Café,Juice Bar,Italian Restaurant,French Restaurant,Hotel Bar
35,LONDON,6,Hotel,Coffee Shop,Theater,Pub,Burger Joint,Café,Juice Bar,Italian Restaurant,French Restaurant,Hotel Bar
49,LONDON,6,Hotel,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Pub,Garden,Wine Bar,Falafel Restaurant,Restaurant,Scenic Lookout
68,LONDON,6,Clothing Store,Plaza,Ice Cream Shop,Bookstore,Restaurant,Boutique,Italian Restaurant,Coffee Shop,Bakery,Gym / Fitness Center


This concludes the review of our London cluster dataset 

# 3.3 MAD Section

3.3.1 Data Scraping

In [147]:
url_mad = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
wiki_mad_url = requests.get(url_mad)
wiki_mad_url

<Response [200]>

In [148]:
wiki_mad_data = pd.read_html(wiki_mad_url.text)
wiki_mad_data

[    0                                                  1
 0 NaN  This article possibly contains original resear...,
     District Number                 Name  District area[n 1] (Ha.)  \
 0               1.0               Centro                    522.82   
 1               2.0           Arganzuela                    646.22   
 2               3.0               Retiro                    546.62   
 3               4.0            Salamanca                    539.24   
 4               5.0            Chamartín                    917.55   
 5               6.0               Tetuán                    537.47   
 6               7.0             Chamberí                    467.92   
 7               8.0  Fuencarral-El Pardo                  23783.84   
 8               9.0      Moncloa-Aravaca                   4653.11   
 9              10.0               Latina                   2542.72   
 10             11.0          Carabanchel                   1404.83   
 11             12.0           

In [149]:
wiki_mad_data = wiki_mad_data[1]
wiki_mad_data

,District Number,Name,District area[n 1] (Ha.),Population,Population density(Hab./Ha.),Location,Administrative wards
0,1.0,Centro,522.82,131928,252.34,NaN,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,2.0,Arganzuela,646.22,151965,235.16,NaN,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,3.0,Retiro,546.62,118516,216.82,NaN,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,4.0,Salamanca,539.24,143800,266.67,NaN,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,5.0,Chamartín,917.55,143424,156.31,NaN,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,6.0,Tetuán,537.47,153789,286.13,NaN,Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,7.0,Chamberí,467.92,137401,293.64,NaN,Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...
7,8.0,Fuencarral-El Pardo,23783.84,238756,10.04,NaN,El Pardo (81)Fuentelarreina (82)Peñagrande (83...
8,9.0,Moncloa-Aravaca,4653.11,116903,25.12,NaN,Casa de Campo (91)Argüelles (92)Ciudad Univers...
9,10.0,Latina,2542.72,233808,91.95,NaN,Los Cármenes (101)Puerta del Ángel (102)Lucero...


3.3.2 Data - Area List Processing 

In [150]:
wiki_mad_data.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
wiki_mad_data

,District_Number,Name,District_area[n_1]_(Ha.),Population,Population_density(Hab./Ha.),Location,Administrative_wards
0,1.0,Centro,522.82,131928,252.34,NaN,Palacio (11)Embajadores (12)Cortes (13)Justici...
1,2.0,Arganzuela,646.22,151965,235.16,NaN,Imperial (21)Acacias (22)Chopera (23)Legazpi (...
2,3.0,Retiro,546.62,118516,216.82,NaN,Pacífico (31)Adelfas (32)Estrella (33)Ibiza (3...
3,4.0,Salamanca,539.24,143800,266.67,NaN,Recoletos (41)Goya (42)Fuente del Berro (43)Gu...
4,5.0,Chamartín,917.55,143424,156.31,NaN,El Viso (51)Prosperidad (52)Ciudad Jardín (53)...
5,6.0,Tetuán,537.47,153789,286.13,NaN,Bellas Vistas (61)Cuatro Caminos (62)Castillej...
6,7.0,Chamberí,467.92,137401,293.64,NaN,Gaztambide (71)Arapiles (72)Trafalgar (73)Alma...
7,8.0,Fuencarral-El Pardo,23783.84,238756,10.04,NaN,El Pardo (81)Fuentelarreina (82)Peñagrande (83...
8,9.0,Moncloa-Aravaca,4653.11,116903,25.12,NaN,Casa de Campo (91)Argüelles (92)Ciudad Univers...
9,10.0,Latina,2542.72,233808,91.95,NaN,Los Cármenes (101)Puerta del Ángel (102)Lucero...


In [151]:
df2 = wiki_mad_data.drop( [ wiki_mad_data.columns[0], wiki_mad_data.columns[2], wiki_mad_data.columns[3], wiki_mad_data.columns[4], wiki_mad_data.columns[5], wiki_mad_data.columns[6] ], axis=1)

In [152]:
df2.head()

,Name
0,Centro
1,Arganzuela
2,Retiro
3,Salamanca
4,Chamartín


In [153]:
df2.columns=['Zone']
df2

,Zone
0,Centro
1,Arganzuela
2,Retiro
3,Salamanca
4,Chamartín
5,Tetuán
6,Chamberí
7,Fuencarral-El Pardo
8,Moncloa-Aravaca
9,Latina


In [154]:
df3 = df2[df2.index != 21]

3.3.3 Data - Geolocating and Map Visualisation

In [156]:
def get_x_y_spn(address1):
   lat_coords = 0
   lng_coords = 0
   g = geocode(address='{}, Madrid, Spain'.format(address1))[0]
   lng_coords = g['location']['x']
   lat_coords = g['location']['y']
   return str(lat_coords) +","+ str(lng_coords)

In [157]:
geo_xy_spn = df3['Zone']    
geo_xy_spn

0                  Centro
1              Arganzuela
2                  Retiro
3               Salamanca
4               Chamartín
5                  Tetuán
6                Chamberí
7     Fuencarral-El Pardo
8         Moncloa-Aravaca
9                  Latina
10            Carabanchel
11                  Usera
12     Puente de Vallecas
13              Moratalaz
14          Ciudad Lineal
15              Hortaleza
16             Villaverde
17      Villa de Vallecas
18              Vicálvaro
19    San Blas-Canillejas
20                Barajas
Name: Zone, dtype: object

In [158]:
lalnspn = geo_xy_spn.apply(lambda x: get_x_y_spn(x))
lalnspn

0       41.62812000000008,-4.727049999999963
1        40.40021000000007,-3.69617999999997
2      40.41317000000004,-3.6830699999999297
3       40.42972000000003,-3.679749999999956
4      40.450000000000045,-3.699999999999932
5       40.46667000000008,-3.716669999999965
6      40.43404000000004,-3.7037899999999695
7       40.49840000000006,-3.731399999999951
8     40.435470000000066,-3.7316999999999325
9      40.38897000000003,-3.7456899999999678
10     40.39094000000006,-3.7241999999999393
11     40.38866000000007,-3.7003499999999576
12       40.39235000000008,-3.65969999999993
13     40.41667000000007,-3.6499999999999773
14    40.453490000000045,-3.6543399999999338
15     40.47444000000007,-3.6410999999999376
16      40.35000000000008,-3.699999999999932
17     40.36695000000003,-3.6014599999999746
18     40.400000000000034,-3.599999999999966
19     40.43450993060772,-3.6098999744647267
20     40.491810000000044,-3.569479999999942
Name: Zone, dtype: object

In [159]:
laspn = lalnspn.apply(lambda x: x.split(',')[0])
laspn

0      41.62812000000008
1      40.40021000000007
2      40.41317000000004
3      40.42972000000003
4     40.450000000000045
5      40.46667000000008
6      40.43404000000004
7      40.49840000000006
8     40.435470000000066
9      40.38897000000003
10     40.39094000000006
11     40.38866000000007
12     40.39235000000008
13     40.41667000000007
14    40.453490000000045
15     40.47444000000007
16     40.35000000000008
17     40.36695000000003
18    40.400000000000034
19     40.43450993060772
20    40.491810000000044
Name: Zone, dtype: object

In [160]:
lnspn = lalnspn.apply(lambda x: x.split(',')[1])
lnspn

0      -4.727049999999963
1       -3.69617999999997
2     -3.6830699999999297
3      -3.679749999999956
4      -3.699999999999932
5      -3.716669999999965
6     -3.7037899999999695
7      -3.731399999999951
8     -3.7316999999999325
9     -3.7456899999999678
10    -3.7241999999999393
11    -3.7003499999999576
12      -3.65969999999993
13    -3.6499999999999773
14    -3.6543399999999338
15    -3.6410999999999376
16     -3.699999999999932
17    -3.6014599999999746
18     -3.599999999999966
19    -3.6098999744647267
20     -3.569479999999942
Name: Zone, dtype: object

In [161]:
spn_mrg = pd.concat([df3,laspn.astype(float), lnspn.astype(float)], axis=1)
spn_mrg.columns= ['Zone','latitude','longitude']
spn_mrg

,Zone,latitude,longitude
0,Centro,41.62812,-4.72705
1,Arganzuela,40.40021,-3.69618
2,Retiro,40.41317,-3.68307
3,Salamanca,40.42972,-3.67975
4,Chamartín,40.45000,-3.70000
5,Tetuán,40.46667,-3.71667
6,Chamberí,40.43404,-3.70379
7,Fuencarral-El Pardo,40.49840,-3.73140
8,Moncloa-Aravaca,40.43547,-3.73170
9,Latina,40.38897,-3.74569


In [162]:
spn = geocode(address='Madrid, Spain')[0]
spn_ln_coords = spn ['location']['x']
spn_la_coords = spn['location']['y']

# Map of Madrid with Neighbourhoods

In [163]:
map_spn = folium.Map(location=[spn_la_coords, spn_ln_coords], zoom_start=12)
map_spn

for latitude, longitude, Zone in zip (spn_mrg['latitude'], spn_mrg['longitude'], spn_mrg['Zone']):
    label = '{}'.format(Zone)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='red',
        fill=True
        ).add_to(map_spn)  
    
map_spn

3.3.4 Data - Venue Extraction and Grouping


In [164]:
CLIENT_ID = 'DDJ52YS2BFQJWYWRIMTCIYEERAAHV3KRB5CE3SSHRAW4CDCU' 
CLIENT_SECRET = 'MKF5M1HHI3XHYMYFJ0STACUUAIJ31HTTS1Y3P5K4BMJEBCDO'
VERSION = '20180605'

In [165]:
LIMIT=50

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT
            )
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

In [166]:
spnvenue = getNearbyVenues(spn_mrg['Zone'], spn_mrg['latitude'], spn_mrg['longitude'])

Centro
Arganzuela
Retiro
Salamanca
Chamartín
Tetuán
Chamberí
Fuencarral-El Pardo
Moncloa-Aravaca
Latina
Carabanchel
Usera
Puente de Vallecas
Moratalaz
Ciudad Lineal
Hortaleza
Villaverde
Villa de Vallecas
Vicálvaro
San Blas-Canillejas
Barajas


In [167]:
spnvenue.head()
#Data QC

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Category
0,Centro,41.62812,-4.72705,Brasería La Berrea,Spanish Restaurant
1,Centro,41.62812,-4.72705,Moby Dick,Pub
2,Centro,41.62812,-4.72705,Casetas Fiestas Delicias,Tapas Restaurant
3,Centro,41.62812,-4.72705,Zeleris,Shipping Store
4,Centro,41.62812,-4.72705,Lara,Bar


In [168]:
spnvenue.groupby('Venue Category').max()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue
Venue Category,,,,
Airport Lounge,Barajas,40.49181,-3.56948,Sala VIP Neptuno
American Restaurant,Salamanca,40.43404,-3.65970,Rochela Restaurante
Arepa Restaurant,Arganzuela,40.40021,-3.69618,Patacón Pisao
Argentinian Restaurant,Usera,40.45349,-3.65434,La Vaca Argentina
Art Gallery,Usera,40.45000,-3.67975,Matadero Madrid
...,...,...,...,...
Train Station,Villaverde,40.40021,-3.60000,Cercanías Vicálvaro
Udon Restaurant,Villa de Vallecas,40.36695,-3.60146,UDON
Video Game Store,Villa de Vallecas,40.36695,-3.60146,Game


In [169]:
spnvenuecat = pd.get_dummies(spnvenue[['Venue Category']], prefix="", prefix_sep="")
spnvenuecat

,Airport Lounge,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
437,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [170]:
spnvenuecat['Neighbourhood'] = spnvenue['Neighbourhood'] 

fixed_columns = [spnvenuecat.columns[-1]] + list(spnvenuecat.columns[:-1])
spnvenuecat = spnvenuecat[fixed_columns]

spnvenuecat.head()

,Neighbourhood,Airport Lounge,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Centro,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Centro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Mean values of Categories - Mean of venue catagories calculated by Neighbourhood - MAD

In [171]:
spnall = spnvenuecat.groupby('Neighbourhood').mean().reset_index()
spnall.head()

,Neighbourhood,Airport Lounge,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Toy / Game Store,Trade School,Train Station,Udon Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,Arganzuela,0.000000,0.0,0.04,0.0,0.00,0.0,0.0,0.0,0.0,...,0.100000,0.02,0.02,0.0,0.02,0.02,0.0,0.0,0.0,0.00
1,Barajas,0.384615,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00
2,Carabanchel,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00
3,Centro,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,...,0.166667,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.00
4,Chamartín,0.000000,0.0,0.00,0.0,0.02,0.0,0.0,0.0,0.0,...,0.040000,0.00,0.04,0.0,0.00,0.00,0.0,0.0,0.0,0.02


In [172]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [173]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create Top Venue Categories - MAD

In [174]:
spnvensrtnh = pd.DataFrame(columns=columns)
spnvensrtnh['Neighbourhood'] = spnall ['Neighbourhood']

for ind in np.arange(spnall.shape[0]):
    spnvensrtnh.iloc[ind, 1:] = return_most_common_venues(spnall.iloc[ind, :], num_top_venues)

spnvensrtnh.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arganzuela,Tapas Restaurant,Spanish Restaurant,Restaurant,Bakery,Grocery Store,Arepa Restaurant,Beer Garden,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
1,Barajas,Airport Lounge,Spanish Restaurant,Juice Bar,Coffee Shop,Fast Food Restaurant,Sporting Goods Shop,Duty-free Shop,Breakfast Spot,Farmers Market,Flea Market
2,Carabanchel,Coffee Shop,Grocery Store,Plaza,Metro Station,Fast Food Restaurant,Farmers Market,Supermarket,Flea Market,Food,Duty-free Shop
3,Centro,Tapas Restaurant,Shipping Store,Spanish Restaurant,Pub,Diner,Bar,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Chamartín,Spanish Restaurant,Bakery,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Clothing Store,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Gift Shop


3.3.5 Modelling - K Means 

In [176]:
k_num_clusters = 6

spnallclstr = spnall.drop('Neighbourhood', 1)

spnkm = KMeans(n_clusters=k_num_clusters, random_state=0).fit(spnallclstr)
spnkm

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=6, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [177]:
spnkm.labels_

array([0, 2, 4, 0, 0, 0, 0, 0, 4, 5, 1, 0, 0, 0, 0, 0, 0, 3, 0, 3])

In [178]:
spnvensrtnh.insert(0, 'Cluster Labels', spnkm.labels_ +1)

In [179]:
spndata = spn_mrg

spndata = spndata.join(spnvensrtnh.set_index('Neighbourhood'), on='Zone')

spndata.head()

,Zone,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,41.62812,-4.72705,1.0,Tapas Restaurant,Shipping Store,Spanish Restaurant,Pub,Diner,Bar,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,Arganzuela,40.40021,-3.69618,1.0,Tapas Restaurant,Spanish Restaurant,Restaurant,Bakery,Grocery Store,Arepa Restaurant,Beer Garden,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
2,Retiro,40.41317,-3.68307,1.0,Garden,Café,Plaza,Monument / Landmark,Fountain,Snack Place,Paella Restaurant,Outdoor Sculpture,Coffee Shop,Diner
3,Salamanca,40.42972,-3.67975,1.0,Spanish Restaurant,Restaurant,Boutique,Mediterranean Restaurant,Tapas Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Playground,Plaza
4,Chamartín,40.45000,-3.70000,1.0,Spanish Restaurant,Bakery,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Clothing Store,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Gift Shop


In [180]:
spndatanan = spndata.dropna(subset=['Cluster Labels'])

3.3.6 Data Analyis

In [181]:
map_clusters_madrid = folium.Map(location=[spn_ln_coords, spn_la_coords], zoom_start=12)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(spndata['latitude'], spndata['longitude'], spndata['Zone'], spndata['Cluster Labels']):
    label = folium.Popup('Cluster Label' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters_madrid)
        
map_clusters_madrid

ValueError: cannot convert float NaN to integer

In [182]:
spndatanan.loc[spndata['Cluster Labels'] == 1, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,Tapas Restaurant,Shipping Store,Spanish Restaurant,Pub,Diner,Bar,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
1,Arganzuela,Tapas Restaurant,Spanish Restaurant,Restaurant,Bakery,Grocery Store,Arepa Restaurant,Beer Garden,Falafel Restaurant,Gym / Fitness Center,Chinese Restaurant
2,Retiro,Garden,Café,Plaza,Monument / Landmark,Fountain,Snack Place,Paella Restaurant,Outdoor Sculpture,Coffee Shop,Diner
3,Salamanca,Spanish Restaurant,Restaurant,Boutique,Mediterranean Restaurant,Tapas Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Playground,Plaza
4,Chamartín,Spanish Restaurant,Bakery,Japanese Restaurant,Italian Restaurant,Thai Restaurant,Clothing Store,Tapas Restaurant,Restaurant,Mediterranean Restaurant,Gift Shop
5,Tetuán,Spanish Restaurant,Soccer Field,Park,Asian Restaurant,Supermarket,Electronics Store,Flea Market,Fast Food Restaurant,Farmers Market,Falafel Restaurant
6,Chamberí,Spanish Restaurant,Restaurant,Café,Tapas Restaurant,Brewery,Ice Cream Shop,Plaza,Multiplex,Bakery,Burger Joint
11,Usera,Spanish Restaurant,Gastropub,Clothing Store,Coffee Shop,Beer Garden,Italian Restaurant,Park,Restaurant,Pub,Plaza
12,Puente de Vallecas,Pizza Place,Motorcycle Shop,Rock Club,Soccer Stadium,Restaurant,Electronics Store,Music Venue,Supermarket,Brewery,Big Box Store
14,Ciudad Lineal,Restaurant,Chinese Restaurant,Spanish Restaurant,Pharmacy,Café,Burger Joint,Boutique,Food Truck,Cosmetics Shop,Park


In [183]:
spndatanan.loc[spndata['Cluster Labels'] == 2, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Moratalaz,Pool,BBQ Joint,Yoga Studio,Food Truck,Cuban Restaurant,Department Store,Dessert Shop,Diner,Donut Shop,Duty-free Shop


In [184]:
spndatanan.loc[spndata['Cluster Labels'] == 3, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Barajas,Airport Lounge,Spanish Restaurant,Juice Bar,Coffee Shop,Fast Food Restaurant,Sporting Goods Shop,Duty-free Shop,Breakfast Spot,Farmers Market,Flea Market


In [185]:
spndatanan.loc[spndata['Cluster Labels'] == 4, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Villaverde,Train Station,Spanish Restaurant,Grocery Store,Gastropub,Mediterranean Restaurant,Yoga Studio,Electronics Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant
18,Vicálvaro,Grocery Store,Train Station,Metro Station,Falafel Restaurant,Electronics Store,Duty-free Shop,Flea Market,Fast Food Restaurant,Farmers Market,Yoga Studio


In [186]:
spndatanan.loc[spndata['Cluster Labels'] == 5, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Latina,Fast Food Restaurant,Grocery Store,Indie Theater,Metro Station,Plaza,Comedy Club,Breakfast Spot,Spanish Restaurant,Pizza Place,General Entertainment
10,Carabanchel,Coffee Shop,Grocery Store,Plaza,Metro Station,Fast Food Restaurant,Farmers Market,Supermarket,Flea Market,Food,Duty-free Shop


In [187]:
spndatanan.loc[spndata['Cluster Labels'] == 6, spndatanan.columns[[0] + list(range(4, spndatanan.shape[1]))]]

,Zone,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Moncloa-Aravaca,Pool,Pub,Gym,Spanish Restaurant,Yoga Studio,Donut Shop,Farmers Market,Falafel Restaurant,Electronics Store,Duty-free Shop


# Results and Discussion

This review has utilised regional data from Wikipedia and extracted city zonal data combined with venue sourced data from FourSquare. 

THe results suggest that if you like Pools, Madrid is the place to go with a higher number than London. Also for Tapas and Spanish cuisine Madrid is far better than London. Trains / subway in Madrid appears the most frequently common transport venue. Supermarkets are fairly common in most zones.

In London Hotels and Pubs are generally the most frequent venues with a range of restaurants (including Tapas and many more). Transport in London is dominated by Subway with a higher referenced propostion of Bus Stations relative to Madrid. Pools are low down the list of occurrences unfortunately. 

In summary, London is a fairly cospompolitan location, however Madrid has a very authentic Spanish culture and a far better climate. If you like swimming, Madrid also seems the one to be. 

# Conclusion

This review has presented the venue and zone data from the cities of Madrid and London. Both vibrant cities with different things to offer. 

If you are looking for swimming pools Madrid is the place to go, if you are looking for a range of restaurants and hotels, then London is the best bet. 

Recommendations. 
In future further cluster analysis could be undertaken looking at more refined zones in specific tourist areas rather than the whole city.